# Basics of Spark DataFrame
___
### Import Pyspark and Check Version

In [1]:
import pyspark
pyspark.__version__

'2.4.5'

### Create Spark Session

In [2]:
# Import SparkSession
from pyspark.sql import SparkSession

In [3]:
# Initiate SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [4]:
spark

### Read Data to DataFrame

In [5]:
df = spark.read.json('dataset/people.json')

In [6]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Show Dataset Schema

In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



### Show Columns

In [8]:
df.columns

['age', 'name']

### Show Descriptive Statistics

In [9]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



___
# Read DataFrame with Infer Schema Manualy
Sometimes Spark cannot infer the Data type of the dataset, but we can manually assign datatype to the schema

In [10]:
from pyspark.sql.types import (StructField, StructType,
                               IntegerType, StringType)

In [11]:
# Create schema and assign type to StructField
data_schema = [StructField('age', IntegerType(), nullable=True),
               StructField('name', StringType(), nullable=True)]

# Create Struct type from data_schema
final_struct = StructType(fields=data_schema)

In [12]:
# Load Dataset using custom schema
df = spark.read.json('dataset/people.json', schema=final_struct)

# Print the Schema of DataFrame
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



___

# Select DataFrame Subset

## Select columns

```python
df.show()
```

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>age</th>      <th>name</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>NaN</td>      <td>Michael</td>    </tr>    <tr>      <th>1</th>      <td>30.0</td>      <td>Andy</td>    </tr>    <tr>      <th>2</th>      <td>19.0</td>      <td>Justin</td>    </tr>  </tbody></table>

Usually, in Pandas, we do subset or selecting only some columns from dataframe by indexing `df['columns']`

In [13]:
df['age']

Column<b'age'>

In [14]:
type(df['age'])

pyspark.sql.column.Column

### Using .select() method

It will return pyspark Column object, to subset the columns, we can use method **`df.select()`**

In [15]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



## Select Rows object
Now, let's try to select only top 2 rows from data using **`.head()`** and passing argument **2**

In [16]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

We can also index the list using `[]`

In [17]:
df.head(2)[1]

Row(age=30, name='Andy')

Showing Rows object type

In [18]:
type(df.head(2)[0])

pyspark.sql.types.Row

## Select multiple columns 

In [19]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Create New column
TO create new columns, we can use method **`.withColumn()`**

In [20]:
df.withColumn('new_age', df['age'] * 2).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     60|
|  19| Justin|     38|
+----+-------+-------+



But those operation is not _**inplace**_ operator, so the changes are not permanent

In [21]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Rename Columns

In [22]:
df.withColumnRenamed('age', 'new_age').show()

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



## Using SQL Syntax
Spark has also capability to use SQL syntax to operate with dataframe.

But first, we need to register the dataframe as SQL temporary view

In [23]:
# create temporary view `people` from dataframe
df.createOrReplaceTempView('people')

In [24]:
# Example 1
result = spark.sql("SELECT * FROM people")

result.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [25]:
# Example 2
new_result = spark.sql("SELECT * FROM people WHERE age > 10")

new_result.show()

+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
+---+------+

